## Load and transform simulation outputs.

In [ ]:
from shared.preface import *
import shared.functions as fct

def transform_simulation_outputs(box_name, box_ver, sim_fullname):
    """
    Input sim_fullname specifies the simulation mode with additional information: e.g. "single_halos_40k_test1" or "spheres_5shells_10k_final".
    """

    # Path to simulation outputs.
    out_dir = f'{box_name}/{box_ver}/{sim_fullname}'

    if 'single_halos' or 'spheres' in sim_fullname:
        # Load neutrino vectors (positions and velocities) of the 10k batches, and concatenate them into a single array.
        neutrino_vectors = np.array([[[]]])  # create empty (1,1,1) array

        batch_paths = glob.glob(f'{out_dir}/neutrino_vectors_batch*.npy')
        for batch_path in batch_paths:
            batch_vectors = np.load(batch_path)
            neutrino_vectors = np.concatenate(
                (neutrino_vectors, batch_vectors), axis=1
            )

        # Calculate overdensities.
        overdensities = np.load(f'{out_dir}/number_densities.npy')/N0

        return overdensities, neutrino_vectors
    
    else:
        # Load angle pairs and calculate overdensities for the all_sky mode.
        all_sky_output = np.load(f'{out_dir}/number_densities.npy')
        angle_pairs = all_sky_output[:, :2]
        overdensities = all_sky_output[:, 2:]/N0

        return overdensities, angle_pairs
    

box_name = 'L025N752'
box_ver = 'DMONLY/SigmaConstant00'
sim_fullname = 'single_halos_10k'
sim_arrays = transform_simulation_outputs(box_name, box_ver, sim_fullname)

## Analyze and plot simulation outputs.

In [ ]:
def analyze_simulation_outputs(
        sim_arrays_to_analyze, 
        box_name, box_ver, sim_fullname,
        plots_to_make
    ):

    # Figure directory.
    if not os.path.exists('figures'):
        os.makedirs('figures')
    fig_dir = f'figures/{box_name}/{box_ver}/{sim_fullname}'

    savefig_args = dict(
        bbox_inches='tight'
    )


    ### ================================== ###
    ### Figure of merit: overdensity band. ###
    ### ================================== ###

    fig, ax = plt.subplots(1,1)

    # Numerical simulation.
    if etas_sim.ndim <= 1:
        ax.plot(
            m_nu_range*1e3, (etas_sim-1), color='blue', 
            label='medians'
        )
    else:
        nus_median = np.median(etas_sim, axis=0)
        nus_perc2p5 = np.percentile(etas_sim, q=2.5, axis=0)
        nus_perc97p5 = np.percentile(etas_sim, q=97.5, axis=0)
        nus_perc16 = np.percentile(etas_sim, q=16, axis=0)
        nus_perc84 = np.percentile(etas_sim, q=84, axis=0)
        ax.plot(
            m_nu_range*1e3, (nus_median-1), color='blue', 
            label='Halo sample medians'
        )
        ax.fill_between(
            m_nu_range*1e3, (nus_perc2p5-1), (nus_perc97p5-1), 
            color='blue', alpha=0.2, label='2.5-97.5 %'
        )
        ax.fill_between(
            m_nu_range*1e3, (nus_perc16-1), (nus_perc84-1), 
            color='blue', alpha=0.3, label='16-84 %'
        )
    plt.savefig(fig_dir, **savefig_args)